In [1]:
! pip install langchain-chroma # pour le vector database

  Using cached build-1.2.1-py3-none-any.whl.metadata (4.3 kB)
     ---------------------------------------- 67.3/67.3 kB 1.2 MB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Installing backend dependencies: started
  Installing backend dependencies: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Using cached pyproject_hooks-1.1.0-py3-none-any.whl.metadata (1.3 kB)
INFO: pip is looking at multiple versions of opentelemetry-sdk to determine which version is compatible with other requirements. This could take a while.
   ---------------------------------------- 526.8/526.8 kB 1.9 MB/s eta 0:00:00
   ---------------------------------------- 150.6/150.6 kB 1.8 MB/s eta 0:00:00
   ---------------------------------

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain 0.1.13 requires langchain-core<0.2.0,>=0.1.33, but you have langchain-core 0.2.5 which is incompatible.
langchain-community 0.0.29 requires langchain-core<0.2.0,>=0.1.33, but you have langchain-core 0.2.5 which is incompatible.
langchain-text-splitters 0.0.1 requires langchain-core<0.2.0,>=0.1.28, but you have langchain-core 0.2.5 which is incompatible.


In [2]:
! pip install langchain-core sentence-transformers

In [3]:
! pip install chromadb

In [4]:
! pip install pypdf

   ---------------------------------------- 290.4/290.4 kB 2.2 MB/s eta 0:00:00


In [7]:
! pip install langchain-community

   ---------------------------------------- 302.9/302.9 kB 1.7 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.2.5
    Uninstalling langchain-core-0.2.5:
      Successfully uninstalled langchain-core-0.2.5


langchain-chroma # pour le vector database  \
langchain-core # pour les parseurs \
sentences-transformers # pour les modèles d'embedding \
chromadb # pour le vector databse \
pypdf # pour parser les pdf

In [1]:
from langchain_community.document_loaders import PyPDFLoader

In [2]:
loader = PyPDFLoader('./code_civil.pdf')

In [3]:
pages = loader.load_and_split()

In [6]:
for page in pages[:3]:
    print(page.metadata)

{'source': './code_civil.pdf', 'page': 0}
{'source': './code_civil.pdf', 'page': 1}
{'source': './code_civil.pdf', 'page': 2}


In [10]:
pages_filtered = list(filter(lambda page: (p := page.metadata['page']) >= 12 and p<=655, pages))

In [14]:
pages_filtered[13]

Document(page_content="Livre Ier : Des personnes - Titre Ier bis : De la nationalité française - Chapitre II : De la nationalité française d'origine\n19-3  Ordonnance n°2005-759 du 4 juillet 2005 - art. 17 () JORF 6 juillet 2005 en vigueur le 1er juillet 2006      \n Legif.   \n Plan   \n Jp.C.Cass.   \n Jp.Appel   \n Jp.Admin.   \n Juricaf\nEst français l'enfant né en France lorsque l'un de ses parents au moins y est lui-même né.\n19-4  Loi n°93-933 du 22 juillet 1993 - art. 50 () JORF 23 juillet 1993      \n Legif.   \n Plan   \n Jp.C.Cass.   \n Jp.Appel   \n Jp.Admin.   \n Juricaf\nToutefois, si un seul des parents est né en France, l'enfant français, en vertu de l'article 19-3, a la faculté de\nrépudier cette qualité dans les six mois précédant sa majorité et dans les douze mois la suivant.\nCette faculté se perd si l'un des parents acquiert la nationalité française durant la minorité de l'enfant.\nSection 3 : Dispositions communes\n20  Loi n°93-933 du 22 juillet 1993 - art. 50 () 

In [ ]:
from typing import List, Dict, Any

In [ ]:
class Paragraph:
    def __init__(self,
                 number: int,
                 title: str,
                 start_page: int,
                 end_page: int,
                 metadata: Dict[str, Any]):
        self.text = text
        self.metadata = metadata

In [ ]:
class Section:

    def __init__(self,
                 number: int,
                 title: str,
                 paragraphs: 
                 
                 metadata: Dict[str, Any], content: str):
        self.metadata = metadata
        self.content = content

In [ ]:
class Chapitre:

    def __init__(self,
                 number: int,
                 title: str,
                 first_page: int,
                 last_page:int,
                 sections: List[Dict[str, Any]]):
        
        self.title = title
        self.number = number
        self.sections = sections

In [ ]:
class Livre:
    def __init__(self, number: int, title: str, first_page: int, last_page: int, children: List[]):
        self.number = number
        self.title = title
        self.first_page = first_page
        self.last_page = last_page
        self.children = children or []

    

In [15]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [17]:
splitter = RecursiveCharacterTextSplitter(chunk_size=200,
                                              chunk_overlap=50,
                                              length_function=len)


In [18]:
doc_splitted = splitter.split_documents(pages_filtered)

In [19]:
len(doc_splitted)

13426

In [22]:
from langchain_chroma import Chroma
from langchain.embeddings.huggingface import HuggingFaceEmbeddings

In [27]:
embedding_model = HuggingFaceEmbeddings(
    model_name='intfloat/multilingual-e5-small',
    model_kwargs={'device': 'cuda'},
    encode_kwargs={'normalize_embeddings': True},
    cache_folder= '.'
)

modules.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

c:\Users\RAZER\AppData\Local\Programs\Python\Python310\lib\site-packages\huggingface_hub\file_download.py:149: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\RAZER\Desktop\Assistant-Juridique\models--intfloat--multilingual-e5-small. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


README.md:   0%|          | 0.00/160k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/655 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/443 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/167 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/200 [00:00<?, ?B/s]

In [30]:
chroma = Chroma.from_documents(doc_splitted,
                               embedding_model,
                               persist_directory='./chroma-db')

In [ ]:
chroma 

In [32]:
! pip install cohere

  Using cached tokenizers-0.15.2-cp310-none-win_amd64.whl.metadata (6.8 kB)
  Using cached urllib3-2.2.1-py3-none-any.whl.metadata (6.4 kB)
   ---------------------------------------- 165.1/165.1 kB 1.4 MB/s eta 0:00:00
   ---------------------------------------- 139.3/139.3 kB 2.7 MB/s eta 0:00:00
   ---------------------------------------- 497.2/497.2 kB 5.2 MB/s eta 0:00:00
Using cached tokenizers-0.15.2-cp310-none-win_amd64.whl (2.2 MB)
   ---------------------------------------- 12.3/12.3 MB 2.6 MB/s eta 0:00:00
   ---------------------------------------- 82.2/82.2 kB 2.3 MB/s eta 0:00:00
Using cached urllib3-2.2.1-py3-none-any.whl (121 kB)
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.26.18
    Uninstalling urllib3-1.26.18:
      Successfully uninstalled urllib3-1.26.18
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.19.1
    Uninstalling tokenizers-0.19.1:
      Successfully uninstalled tokenizers-0.19.1


  You can safely remove it manually.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
transformers 4.40.1 requires tokenizers<0.20,>=0.19, but you have tokenizers 0.15.2 which is incompatible.


In [33]:
chroma.similarity_search_with_score('droit de la famille', 5)

[(Document(page_content="du droit à l'image de l'enfant.\nDans tous les cas visés au présent article, les deux parents doivent être appelés à l'instance. Lorsque l'enfant", metadata={'page': 166, 'source': './code_civil.pdf'}),
  0.20696397125720978),
 (Document(page_content="faire obstacle à l'exercice de ce droit.\nSi tel est l'intérêt de l'enfant, le juge aux affaires familiales fixe les modalités des relations entre l'enfant et un", metadata={'page': 153, 'source': './code_civil.pdf'}),
  0.2111358493566513),
 (Document(page_content='Dictionnaire du Droit privé\n> Autorité parentale\n> Juge aux affaires familiales (JAF)\n> Bien Propre\n> Séparation de biens\nRessources\n> Les modes de saisine du JAF', metadata={'page': 95, 'source': './code_civil.pdf'}),
  0.21270613372325897),
 (Document(page_content="par l'intérêt de la famille.\nL'acte passé dans les conditions fixées par l'autorisation de justice est opposable à l'époux dont le concours ou", metadata={'page': 94, 'source': './c

In [35]:
from langchain.llms import HuggingFacePipeline
from langchain.prompts import PromptTemplate
from transformers import pipeline
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

In [36]:
text_gen_pipeline = pipeline('text-generation',
                             device='cuda')

No model was supplied, defaulted to openai-community/gpt2 and revision 6c0e608 (https://huggingface.co/openai-community/gpt2).
Using a pipeline without specifying a model name and revision in production is not recommended.


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

c:\Users\RAZER\AppData\Local\Programs\Python\Python310\lib\site-packages\huggingface_hub\file_download.py:149: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\RAZER\.cache\huggingface\hub\models--openai-community--gpt2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [37]:
llm = HuggingFacePipeline(pipeline=text_gen_pipeline)

In [38]:
propmt_template = """
Avec l'aide de ces informations provenant du code civil français, réponds à la question.
Contexte: {context}
Question: {question}
"""

In [39]:
prompt = PromptTemplate(template=propmt_template,
                        input_variables=['context', 'question'])

In [40]:
def prepare_docs(docs):
    return "\n".join([doc.page_content for doc in docs])

In [41]:
retriver = chroma.as_retriever()

In [42]:
chain = (
    {'context': retriver | prepare_docs, 'question': RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [43]:
chain

{
  context: VectorStoreRetriever(tags=['Chroma', 'HuggingFaceEmbeddings'], vectorstore=<langchain_chroma.vectorstores.Chroma object at 0x000002542CEBA020>)
           | RunnableLambda(prepare_docs),
  question: RunnablePassthrough()
}
| PromptTemplate(input_variables=['context', 'question'], template="\nAvec l'aide de ces informations provenant du code civil français, réponds à la question.\nContexte: {context}\nQuestion: {question}\n")
| HuggingFacePipeline(pipeline=<transformers.pipelines.text_generation.TextGenerationPipeline object at 0x000002540131F910>)
| StrOutputParser()

In [45]:
chain.invoke('Quel est le droit de la famille en France?')

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


ValueError: Input length of input_ids is 324, but `max_length` is set to 50. This can lead to unexpected behavior. You should consider increasing `max_length` or, better yet, setting `max_new_tokens`.